In [103]:
import json
import os
import tqdm
import pandas as pd

## I. convert emails text (both training and testing) into appropriate jsonl file format 

### 6088 entries in training set ( 2000+ machine generated, the rest are human-written)
#### 4000+ are from email corpus, 2000+ are from gtp-2 generated and the ENRON Email Dataset 
###### kaggle datasets download -d nitishabharathi/email-spam-dataset

In [50]:
PATH = '/Users/jessicademacbook/DSCI-550-Assignment-2/data/additional-features-v2/new/8_GPT-2_Generated_Text_for_Grover/'
folders = [f for f in os.listdir(PATH) if not f.startswith('.')]

In [51]:
#read all machine txt in each folders, label all machine generated content
lis=[]
for folder in folders:   
    for i in os.listdir(f'{PATH}{folder}'):  
        f=open(f'{PATH}{folder}/{i}','r')
        text=f.read()
        text_dic={"text":text,"label":"machine","split":"train"}
        lis.append(text_dic)

In [52]:
#read all human json in email corpus, label all content as human
path='/Users/jessicademacbook/DSCI-550-Assignment-1/data/separated by email/'
for i in os.listdir(path):
    if i.endswith('.json'):
        f=open(f'{path}{i}','r')
        text=json.load(f)
        try:
            content=text["X-TIKA:content"]
            content_dic={"text":content,"label":"human","split":"train"}
            lis.append(content_dic)
        except KeyError:
            pass 

In [114]:
with open('/Users/jessicademacbook/DSCI-550-Assignment-2/data/additional-features-v2/new/fake_emails.csv', "r") as f:
    result=pd.read_csv(f)
    spam=result['Label']==1
    for i in result[spam]['Body']:
        dic={"text":i,"label":"machine","split":"train"}
        lis.append(dic)

    

In [115]:
len(lis)

6088

In [116]:
#write to a jsonl file with all human and machine generated email content
with open('/Users/jessicademacbook/DSCI-550-Assignment-2/data/input_emails.jsonl','w') as outfile:
    for entry in lis:
        json.dump(entry, outfile)
        outfile.write('\n')

In [117]:
#check the written jsonl file has correct labels
with open('/Users/jessicademacbook/DSCI-550-Assignment-2/data/input_emails.jsonl', "r") as f:
    test=[]
    for l in f:
        item = json.loads(l)
        test.append(item['split'])
print(all(test))

True


### Collect 800 email text, labeled as test, write to jsonl file for discrimination 

In [118]:
with open('/Users/jessicademacbook/DSCI-550-Assignment-2/data/test_file.jsonl','w') as f:
    for entry in a_lis:
        json.dump(entry, f)
        f.write('\n')

In [119]:
#get generated text for grover test
new_path = '/Users/jessicademacbook/DSCI-550-Assignment-2/data/additional-features-v2/new/4_GPT-2_Generated_Text/'
folders = [f for f in os.listdir(new_path) if not f.startswith('.')]
test_lis=[]
for folder in folders:   
    for i in os.listdir(f'{new_path}{folder}'):  
        f=open(f'{new_path}{folder}/{i}','r')
        text=f.read()
        text_dic={"text":text,"split":"test","label":"human"}
        test_lis.append(text_dic)
print(len(test_lis))

#write to jsonl file
with open('/Users/jessicademacbook/DSCI-550-Assignment-2/data/test_input.jsonl','w') as f:
    for entry in test_lis:
        json.dump(entry, f)
        f.write('\n')

800


## II. Interpret Grover output result 

In [101]:
import numpy as np

In [123]:
path='/Users/jessicademacbook/DSCI-550-Assignment-2/data/Grover_input_output/output_test-probs_5.npy'
data_array = np.load(path)
print(data_array[0:10])
a=0
for i in data_array:
    if i[0]>0.9:
        a=a+1
print(a)

[[0.9933362  0.00666374]
 [0.9933362  0.00666374]
 [0.9933362  0.00666374]
 [0.9933362  0.00666374]
 [0.9933362  0.00666374]
 [0.9933362  0.00666374]
 [0.9933362  0.00666374]
 [0.9933362  0.00666374]
 [0.9933362  0.00666374]
 [0.9933362  0.00666374]]
800


#####  I am getting same result for every entries. I've tried using different combinations of training data (e.g. 4000 machine+200 human, 4000 human+3000 machine), but accuracy result is either 0 or 1. Output result probs are all the same. 